In [14]:
import tensorflow as tf

# Define batch size and image size for training
batch_size = 32
image_size = (224, 224)
input_shape = image_size + (3,)

# Load datasets from directories and shuffle them with a seed for reproducibility
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '../PotatoImagesData/Train',
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=123
)
classes = train_ds.class_names

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '../PotatoImagesData/Valid',
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=123
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '../PotatoImagesData/Test',
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=123
)

Found 900 files belonging to 3 classes.
Found 300 files belonging to 3 classes.
Found 300 files belonging to 3 classes.


In [15]:
# Image preprocessing - rescaling pixel values
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255), #normalize 0-255 to 0-1   
])

# Data augmentation - applying random transformations for robustness - Horizontal flip, Vertical flip, Rotate, Contrast, Zoom
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

# Model definition - a series of convolutional and pooling layers
model = tf.keras.Sequential([
    resize_and_rescale,
    data_augmentation,
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape = input_shape),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size = (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size = (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(len(classes), activation = 'softmax'), # softmax -> normalize probability of classes 
])

# Model compilation with Adam optimizer and loss function
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Model training with validation data and early stopping
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    batch_size=batch_size,
    callbacks=[early_stopping],
    verbose = 1
)

# Model evaluation on test data
evaluation_results = model.evaluate(test_ds)
print(f"Test Loss: {evaluation_results[0]}, Test Accuracy: {evaluation_results[1]}")

# Saving the trained model
model.save('potato_disease.keras')

Epoch 1/50
29/29 [==============================] - 38s 1s/step - loss: 1.0968 - accuracy: 0.3944 - val_loss: 1.0850 - val_accuracy: 0.3333
Epoch 2/50
29/29 [==============================] - 35s 1s/step - loss: 0.9283 - accuracy: 0.5233 - val_loss: 0.8850 - val_accuracy: 0.6100
Epoch 3/50
29/29 [==============================] - 35s 1s/step - loss: 0.6948 - accuracy: 0.6822 - val_loss: 0.4803 - val_accuracy: 0.7967
Epoch 4/50
29/29 [==============================] - 38s 1s/step - loss: 0.4685 - accuracy: 0.7822 - val_loss: 0.5480 - val_accuracy: 0.7500
Epoch 5/50
29/29 [==============================] - 37s 1s/step - loss: 0.4017 - accuracy: 0.8267 - val_loss: 0.4664 - val_accuracy: 0.7900
Epoch 6/50
29/29 [==============================] - 40s 1s/step - loss: 0.3234 - accuracy: 0.8722 - val_loss: 0.3088 - val_accuracy: 0.8733
Epoch 7/50
29/29 [==============================] - 39s 1s/step - loss: 0.3012 - accuracy: 0.8944 - val_loss: 0.4376 - val_accuracy: 0.8033
Epoch 8/50
29/29 [==

In [17]:
import tensorflow as tf
model = tf.keras.models.load_model('potato_disease.keras')

In [18]:
import tensorflow as tf
import numpy as np

# Function to convert image file to array
def get_img_array(image_path, target_size=image_size):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=target_size)
    img = tf.keras.preprocessing.image.img_to_array(img)
    return img

# Making predictions using the model
predictions = model.predict(np.array([
    get_img_array("../PotatoImagesData/Train/Potato___Early_blight/bb07a1b7-b9ad-4016-a013-9ff0e4636d4a___RS_Early.B 7156.JPG"),
    get_img_array("../PotatoImagesData/Train/Potato___Late_blight/fd35fb86-6558-40f2-b8ac-6922e2107af5___RS_LB 4095.JPG"),
    get_img_array("../PotatoImagesData/Train/Potato___healthy/Potato_healthy-103-_0_3577.jpg"),
]))

# Printing the predicted classes
for i in predictions:
    print(classes[np.argmax(i)])

1/1 [==============================] - 0s 89ms/step
Potato___Early_blight
Potato___Late_blight
Potato___healthy
